In [2]:
from splinter import Browser
from bs4 import BeautifulSoup as soup
import matplotlib.pyplot as plt
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

NameError: name 'engine' is not defined

In [3]:
browser = Browser('chrome')
url = "https://housesigma.com/listings/watched-area-and-community"
browser.visit(url)
html = browser.html
soup_obj = soup(html, 'html.parser')

In [4]:
browser.execute_script('document.querySelector(".app-btn.hs_btn_login_start").click();')

email_input = WebDriverWait(browser.driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, '.form-input.medium.clear.input input[type="email"]'))
)
email = 'qwerty.kitten.m@gmail.com'
passw = 'Bootcamp2024*'
login = (email) 
email_input.send_keys(login)

In [5]:
password_input = WebDriverWait(browser.driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, '.form-input.medium.clear.input input[type="password"]'))
)
password = (passw) 
password_input.send_keys(password)

In [6]:
button = browser.find_by_css('.app-btn.hs_btn_login_submit_email').first
button.click()

In [7]:
browser.is_element_present_by_css('.pc-listing-card', wait_time=10)
html = browser.html
soup_obj = soup(html, 'html.parser')

In [8]:
import json
import re

data_list = []  

articles = soup_obj.find_all('article', class_='pc-listing-card')

for article in articles:
    data = {}

    # Address
    address_element = article.find('h3', class_='address')
    data['Address'] = address_element.text.strip() if address_element else None

    # Neighbourhood
    text1_element = article.find('span', class_='text1')
    data['Neighbourhood'] = text1_element.text.strip() if text1_element else None

    json_scripts = article.find_all('script', class_='hs-script-home-struct', type='application/ld+json')


    for script in json_scripts:
        json_data = script.string
        if json_data:
            json_dict = json.loads(json_data)
            
            # Long and Lat and floor size
            if 'floorSize' in json_dict:
                data['Floor Size'] = json_dict['floorSize']['value']
            if 'geo' in json_dict:
                data['Latitude'] = json_dict['geo']['latitude']
                data['Longitude'] = json_dict['geo']['longitude']

    # Date 
    date_preview_element = article.find('div', class_='date-preview')
    data['Date of Status'] = date_preview_element.text.strip() if date_preview_element else None

    # Price
    highlight_element = article.find('span', class_='highlight')
    data['Price Listed'] = highlight_element.text.strip() if highlight_element else None
   
    # Status
    status_element = article.select_one('div[class^="status-type"]')
    data['Status'] = status_element.text.strip() if status_element else None

    # Sold Price (if status is "Sold" or "Sold Conditional")
    if data['Status'] in ["Sold", "Sold Conditional"]:
        sold_price_element_special = article.select_one('div.price-area span.special')
        sold_price_element_highlight_special = article.select_one('div.price-area span.highlight.special')

        if sold_price_element_special:
            data['Sold Price'] = sold_price_element_special.text.strip()
        elif sold_price_element_highlight_special:
            data['Sold Price'] = sold_price_element_highlight_special.text.strip()
        else:
            data['Sold Price'] = None
    else:
        data['Sold Price'] = 0
    
    # Type of house
    type_element = article.find('p', class_='type')
    data['Type of House'] = type_element.text.strip() if type_element else None

    p_elements = article.find_all('p')
    for p in p_elements:
        text = p.get_text()

        # Bathroom
        if re.search(r'\b(?:bathroom)\b', text):
            bathroom_info = re.findall(r'\d+', text)
            if bathroom_info:
                data['Bathrooms'] = int(bathroom_info[0])  # Convert to integer if not empty

        # Bedroom
        if re.search(r'\b(?:bedroom)\b', text):
            bedroom_info = re.findall(r'\d+', text)
            if bedroom_info:
                data['Bedrooms'] = int(bedroom_info[0])  # Convert to integer if not empty
        
        # Garage
        if re.search(r'\b(?:garage)\b', text):
            garage_info = re.findall(r'\d+', text)
            if garage_info:
                data['Garage'] = int(garage_info[0])  # Convert to integer if not empty
                
    data_list.append(data)

for i, data in enumerate(data_list, 1):
    print(f"{i}: {data}")


1: {'Address': '403 - 407 Dundas St W , Oakville - Rural Oakville', 'Neighbourhood': 'Rural Oakville', 'Floor Size': 849, 'Latitude': -79.74189, 'Longitude': 43.46815, 'Date of Status': '49 minutes ago', 'Price Listed': '$899,900', 'Status': 'Terminated', 'Sold Price': 0, 'Type of House': 'Condo Apt', 'Bedrooms': 2, 'Bathrooms': 2, 'Garage': 1}
2: {'Address': '109 - 3028 Creekshore Common Dr , Oakville - Rural Oakville', 'Neighbourhood': 'Rural Oakville', 'Floor Size': 1099, 'Latitude': -79.7381001, 'Longitude': 43.4720092, 'Date of Status': '6 hours ago', 'Price Listed': '$849,000', 'Status': 'For Sale', 'Sold Price': 0, 'Type of House': 'Condo Apt', 'Bedrooms': 2, 'Bathrooms': 2, 'Garage': 1}
3: {'Address': '1277 Redbank Cres , Oakville - College Park', 'Neighbourhood': 'College Park', 'Floor Size': 1299, 'Latitude': -79.70085, 'Longitude': 43.46068, 'Date of Status': '10 hours ago', 'Price Listed': '$848,000', 'Status': 'For Sale', 'Sold Price': 0, 'Type of House': 'Condo Townhouse'

In [10]:
from datetime import datetime, timedelta

def convert_relative_timestamp(relative_timestamp):
    if " hours ago" in relative_timestamp:
        return datetime.now().date()
    elif " days ago" in relative_timestamp:
        days = int(relative_timestamp.split()[0])
        absolute_timestamp = datetime.now() - timedelta(days=days)
        return absolute_timestamp.date()
    else:
        return datetime.now().date()


df = pd.DataFrame(data_list)
df['City'] = df['Address'].str.split(',').str[1].str.split(' - ').str[0]
df['Address'] = df['Address'].str.split(',').str[0]
df['Date of Status'] = df['Date of Status'].apply(convert_relative_timestamp)

df['Sold Price'] = df['Sold Price'].str.replace('$', '').str.replace(',', '').astype(float)
df['Price Listed'] = df['Price Listed'].str.replace('$', '').str.replace(',', '').astype(float)
df

,Address,Neighbourhood,Floor Size,Latitude,Longitude,Date of Status,Price Listed,Status,Sold Price,Type of House,Bedrooms,Bathrooms,Garage,City
0,403 - 407 Dundas St W,Rural Oakville,849,-79.74189,43.468150,2024-02-04,899900.0,Terminated,NaN,Condo Apt,2,2,1,Oakville
1,109 - 3028 Creekshore Common Dr,Rural Oakville,1099,-79.73810,43.472009,2024-02-04,849000.0,For Sale,NaN,Condo Apt,2,2,1,Oakville
2,1277 Redbank Cres,College Park,1299,-79.70085,43.460680,2024-02-04,848000.0,For Sale,NaN,Condo Townhouse,3,2,0,Oakville
3,2158 Lillykin St,River Oaks,4250,-79.70930,43.478430,2024-02-04,2769900.0,Sold Conditional,2769900.0,Detached,5,6,2,Oakville
4,429 - 2485 Taunton Rd,Uptown Core,649,-79.71938,43.486300,2024-02-04,635000.0,For Sale,NaN,Condo Apt,2,2,1,Oakville
5,2501 - 297 Oak Walk Dr,Uptown Core,549,-79.71817,43.486170,2024-02-04,575000.0,Terminated,NaN,Condo Apt,1,1,1,Oakville
6,118 Tavistock Sq,Old Oakville,2250,-79.68256,43.433340,2024-02-04,2299000.0,For Sale,NaN,Detached,3,2,2,Oakville
7,417 - 185 Robinson St,Old Oakville,1099,-79.66801,43.444960,2024-02-04,773000.0,For Sale,NaN,Condo Apt,2,2,1,Oakville
8,2193 Constance Dr,Eastlake,1750,-79.64955,43.481130,2024-02-04,1850000.0,For Sale,NaN,Detached,4,3,1,Oakville
9,417 - 185 Robinson St,Old Oakville,1099,-79.66801,43.444960,2024-02-04,799500.0,Expired,NaN,Condo Apt,2,2,1,Oakville


In [11]:
import pandas as pd
import numpy as np
from functools import reduce
import requests
import warnings
warnings.filterwarnings('ignore')

In [12]:
weather_data_oakville = 'Resources/Oakville_Historical_Weather.csv'
df_weather = pd.read_csv(weather_data_oakville, encoding = "ISO-8859-1", on_bad_lines = 'skip', low_memory = True, engine = 'python')
df_weather = df_weather.dropna(subset=['MIN_TEMPERATURE'])
df_weather = df_weather.reset_index(drop=True)
df_weather.head()

,x,y,DIRECTION_MAX_GUST,HEATING_DEGREE_DAYS,MIN_TEMPERATURE,MIN_REL_HUMIDITY,PROVINCE_CODE,COOLING_DEGREE_DAYS_FLAG,LOCAL_DATE,STATION_NAME,...,TOTAL_RAIN,MAX_TEMPERATURE,MAX_REL_HUMIDITY,MEAN_TEMPERATURE_FLAG,SPEED_MAX_GUST,TOTAL_SNOW_FLAG,SNOW_ON_GROUND_FLAG,TOTAL_RAIN_FLAG,MIN_REL_HUMIDITY_FLAG,ID
0,-79.696962,43.428058,NaN,6.5,7.0,NaN,ON,NaN,1990-04-20 00:00:00,OAKVILLE GERARD,...,9.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6155PD4.1990.4.20
1,-79.696962,43.428058,NaN,6.7,8.0,NaN,ON,NaN,1990-04-21 00:00:00,OAKVILLE GERARD,...,0.0,14.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6155PD4.1990.4.21
2,-79.696962,43.428058,NaN,5.2,2.0,NaN,ON,NaN,1990-04-22 00:00:00,OAKVILLE GERARD,...,0.0,23.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6155PD4.1990.4.22
3,-79.696962,43.428058,NaN,6.2,7.0,NaN,ON,NaN,1990-04-23 00:00:00,OAKVILLE GERARD,...,0.0,16.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6155PD4.1990.4.23
4,-79.696962,43.428058,NaN,7.7,4.0,NaN,ON,NaN,1990-04-24 00:00:00,OAKVILLE GERARD,...,0.0,16.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6155PD4.1990.4.24


In [3]:
from sqlalchemy import create_engine, MetaData, Table, Column, String, Float, Integer, Date, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session, relationship
from sqlalchemy import create_engine
from datetime import datetime


Base = declarative_base()
#Base.metadata.drop_all(engine)

class House(Base):
    __tablename__ = 'houses'
    id = Column(Integer, primary_key=True)
    address = Column(String)
    status = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    floor_size = Column(Integer)
    bedroom = Column(Integer)
    bathroom = Column(Integer)
    garage = Column(Integer)
    city = Column(String)  
    type = Column(String)
    date_listed = Column(Date)
    neighbourhood = Column(String)

    def __init__(self, address, status, latitude, longitude, floor_size, bedroom, bathroom, garage, city, type, date_listed, neighbourhood):
        self.address = address
        self.status = status
        self.latitude = latitude
        self.longitude = longitude
        self.floor_size = floor_size
        self.bedroom = bedroom
        self.bathroom = bathroom
        self.garage = garage
        self.city = city  
        self.type = type
        self.date_listed = date_listed
        self.neighbourhood = neighbourhood

class Sold(Base):
    __tablename__ = 'sold'
    id = Column(Integer, primary_key=True)
    address = Column(String, ForeignKey('houses.address'))
    price = Column(Float)
    status = Column(String)
    date_listed = Column(Date)
    sold_price = Column(Float)

    def __init__(self, address, city, price, status, date_listed, sold_price):
        self.address = address
        self.price = price
        self.status = status
        self.date_listed = date_listed
        self.sold_price = sold_price

# Define your database engine
#engine = create_engine('sqlite:///oakville_housing.db')


import os

# List of files to delete
files_to_delete = [
    "oakville_housing_sale_data.db",
    "oakville_housing_sale_db.db",
    "oakville_housing_sales_db.db",
    "oakville_housing_sales.db",
    "oakville_housing_html.db",
    "oakville_housing_html.db-journal",
    "oakville_housing.db"
]

# Iterate over the list of files and delete them
for file_name in files_to_delete:
    if os.path.exists(file_name):
        os.remove(file_name)
        print(f"{file_name} deleted successfully")
    else:
        print(f"{file_name} does not exist")

# Optionally, you can also delete any remaining .db-journal files
journal_files = [f for f in os.listdir() if f.endswith(".db-journal")]
for file_name in journal_files:
    os.remove(file_name)
    print(f"{file_name} deleted successfully")

C:\Users\qwert\AppData\Local\Temp\ipykernel_6912\560145780.py:8: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'oakville_housing_sale_data.db'

In [ ]:
for data in data_list:
    address = data.get('Address')
    if address:
        house_exists = session.query(House).filter_by(address=address).first()

    if house_exists:
        house_exists.status = data.get('Status')
        house_exists.date_listed = convert_relative_timestamp(data.get('Date of Status'))
    else:
        # Split the address and assign the city value
        address_parts = address.split(',')
        city = address_parts[1].split(' - ')[0].strip()
        address = address_parts[0]

        new_house = House(
            address=address,
            status=data.get('Status'),
            latitude=data.get('Latitude'),
            longitude=data.get('Longitude'),
            floor_size=data.get('Floor Size'),
            bedroom=data.get('Bedrooms'),
            bathroom=data.get('Bathrooms'),
            garage=data.get('Garage'),
            city=city,  # Assign the city value
            type=data.get('Type of House'),
            date_listed=convert_relative_timestamp(data.get('Date of Status')),
            neighbourhood=data.get('Neighbourhood')
        )
        session.add(new_house)

        # Check if the house is sold
    for data in data_list:
        address = data.get('Address')
        if address:
            house_exists = session.query(House).filter_by(address=address).first()

        if house_exists:
            house_exists.status = data.get('Status')
            house_exists.date_listed = convert_relative_timestamp(data.get('Date of Status'))
        else:
            # Split the address and assign the city value
            address = address_parts[0]

            new_sold_house = Sold(
                address=address,
                price=float(data.get('Price Listed').replace('$', '').replace(',', '')),
                status=data.get('Status'),
                date_listed=convert_relative_timestamp(data.get('Date of Status')),
                sold_price=float(str(data.get('Sold Price')).replace('$', '').replace(',', ''))
            )
            session.add(new_sold_house)


# Commit changes to the database
session.commit()
houses = session.query(House)
sold = session.query(Sold)

for house in houses: 
    print(house.address),
    print(house.neighbourhood),
    print(house.city)

for sale in sold:
    print(sale.price)
    
# Close the session
session.close()
browser.quit()